<a href="https://colab.research.google.com/github/FayeValentain/it-cert-automation-practice/blob/master/Anal%C3%BC%C3%BCsi_praktika.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Toetavad praktilised näited: Andmeteaduse võimalused äriettevõttes
*Praktilised näited tegi Liis Kolberg*

*Materjal baseerub osaliselt Anna Leontjeva loodud materjalil*


<br>

**Selle keskkonna kasutamist selgitavat materjali näete [siit](https://colab.research.google.com/drive/1rq6MywbIT4uiqRuTSLkHvNFChIB5aSzR#forceEdit=true&sandboxMode=true). Videomaterjal on [siin](https://youtu.be/MWeAO4XTWJc).**

<br>


# Sissejuhatus

Käesolevas vihikus vaatame naiivseid näiteid, kuidas praktiliselt andmeteaduse meetodeid andmetele rakendada.
Me alustame kunstlikult genereeritud andmestikuga, mis sisaldab üldistatud informatsiooni klientide tehingutest, millesarnaseid koguvad paljud ettevõtted.

Kasutades seda lähteandmestikku proovime arvutada mõned kirjeldavad statistikud ning visualiseerime tunnuste väärtusi.

See järel rakendame andmetele k-keskmiste algoritmi, et klasterdada kliente kolme tunnuse alusel. Selleks arvutame me need uued tunnused ise: hiljutisus, sagedus ning väärtus, ehk rakendame HSV mudelit.  

Hakkame tööle!

# Andmete sisselugemine

Kõigepealt laadime vajalikud Pythoni paketid ehk teegid

In [ ]:
# andme töötlemisega seotud teegid
import pandas as pd
import numpy as np

# klasterdamisega seotud teegid
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# visualiseerimisega seotud teegid
import matplotlib.pyplot as plt
import seaborn as sns

Kujutle, et oled ettevõtte omanik ja sinu ettevõte on internetis tegutsev lemmikloomapood. Sa oled juba ammu mõelnud, et pead oma parimaid kliente hoidma ning pakkuma parimatele klientidele soodustusi. Sellepärast otsustasid, et on aeg kliente segmenteerida.

Sa oled kokku pannud andmestiku sellest infost, mis sul oli klientide kohta salvestatud. Laeme nüüd selle andmestiku:

In [ ]:
tehingute_andmestik_url = 'https://ndownloader.figshare.com/files/22351839'

In [ ]:
# loeme andmetabeli sisse
andmestik = pd.read_csv(tehingute_andmestik_url, index_col=0)
# vaatame milline see tabel välja näeb (esimesed 5 rida)
andmestik.head()

Vaatame üle, mis on andmestiku dimensioonid ehk kui palju on meil ridu (tehinguid) ning veerge (tunnuseid):

In [ ]:
andmestik.shape

Meil on siin 4181 tehingut ja 4 veergu: tehingu id, tehingu teostamise kuupäev, kliendi id (klient võib teostada mitu tehingut) ja ostusumma.

Vaatame ka, mis andmetüübid erinevates veergudes on.

In [ ]:
andmestik.dtypes

Siin on näha, et *tehingu_id* ja *kliendi_id* tüübid on `int64`, mis tähendab sisuliselt, et need on täisarvulised tunnused. Ostusumma on märgitud kui `float64` tüüpi, mis sisuliselt tähendab, et see tunnus on pidev arvtunnus. Tehingu kuupäeva on väljendatud kui `object`, mis antud juhul tähendab, et kuupäevad on tekstiformaadis. Me peame selle teisendama õigeks kuupäevaformaadiks:

In [ ]:
andmestik['tehingu_kuupäev'] = pd.to_datetime(andmestik['tehingu_kuupäev'])

Kontrollime üle, kas kuupäevade formaat sai edukalt muudetud.

In [ ]:
andmestik.dtypes

Kontrollime ka, kas andmetes esineb puuduvaid väärtusi:

In [ ]:
andmestik.isnull().sum()

Mitte üheski tulbas ei esine puuduvaid väärtusi.

# Andmetega tutvumine ja tunnuste kirjeldamine





Kasulik oleks teada, mis ajavahemikust antud andmed pärinevad, kui paljude erinevate klientide tehingud on kajastatud ning mis on ostusummade väärtuste jaotus.



Vaatame kuupäevade vahemikku:

In [ ]:
# minimaalne kuupäev
andmestik.tehingu_kuupäev.min()

In [ ]:
# maksimaalne kuupäev
andmestik.tehingu_kuupäev.max()

Andmed on vahemikust 4. jaanuar 2013 kuni 31. detsember 2018.

Vaatame nüüd, kui palju unikaalseid kliente meie andmetabelis on.

In [ ]:
andmestik.kliendi_id.nunique()

Andmestik sisaldab 1000 kliendi tehinguid.

Arvutame ka ostusummade jaotuse ülevaateks kirjeldavate statistikute väärtused.

In [ ]:
andmestik.ostusumma.describe()

Näeme, et keskmine ostusumma on umbes 8.07 eurot ja ostusummade väärtused on vahemikust 0.01 ja 38.35 eurot. Mediaanväärtus (tähistatud 50%) on 6.88.

Kuigi väärtuste vahemikud on arvutatud, siis on hea ka neid kuidagi visualiseerida. Kasutame selleks lihtsaid Pythoni pakette, mis pole küll väga ilusad, aga ajavad asja ära. Huvilistel on võimalik ise proovida, kuidas pilte ilusamaks disainida.

Visualiseerime ostusummade väärtusi histogrammiga.

In [ ]:
sns.histplot(data = andmestik, x = "ostusumma", bins = 15)

Histogrammilt on näha, et kõige rohkem on väiksemaid ostusummade väärtusi ja suuri summasid esineb pigem harva.

Vaatame ka karpdiagrammi.

In [ ]:
sns.boxplot(data = andmestik, x = "ostusumma")

Karpdiagrammilt on samuti näha, et ostusummade väärtused on pigem väiksemad (vahemikus 0 kuni 25), aga esineb ka üksikuid suuremaid summasid. Antud juhul ei tundu, et need suuremad summad oleksid kuidagi kahtlaselt anomaalsed (neid on rohkem kui üks) või eriti silmapaistvalt erindid. Antud juhul teeme otsuse neid andmestikust mitte välja jätta.  

Vaatame ka kuidas jagunevad ostusummad läbi aastate. Selleks lisame kõigepealt andmestikku ühe lisaveeru, mis kajastab ainult aastat.

In [ ]:
# tekitame veeru nimega 'aasta'
andmestik['aasta'] = andmestik['tehingu_kuupäev'].dt.strftime('%Y')
# muudame aasta numbriliseks formaadiks, et ta järjestuks piltidel korrektselt
andmestik['aasta'] = pd.to_numeric(andmestik.aasta)
andmestik.head()

Ja koostame nüüd karpdiagrammid läbi aastate.

In [ ]:
sns.boxplot(data = andmestik, y = "ostusumma", x = "aasta")

Näeme, et ostusumma on läbi aastate olnud suhteliselt sama suurusega. Karpdiagrammidega on selline lugu, et nende põhjal ei saa öelda, kui palju erinevaid väärtuseid nad kajastavad.

Vaatamegi, kas ka ostude arv on olnud iga aasta sama. Selleks koostame tulpdiagrammi, mis näitab iga aastast ostude arvu.

In [ ]:
sns.countplot(data = andmestik, x = "aasta")

Tulpdiagrammilt on näha, et ostude arv on iga aasta kasvanud.

Viimasena vaatame igakuist ostude kogusummat joondiagrammiga.

In [ ]:
# summeerime ostusummad kuu kaupa
summ = andmestik.groupby(pd.Grouper(key='tehingu_kuupäev', freq='M')).agg({'ostusumma':'sum'})
summ.head()

In [ ]:
sns.lineplot(data = summ, x = "tehingu_kuupäev", y = "ostusumma", estimator = None)
plt.ylim(0,1000) # teeme kindlaks, et y-telje väärtused algaksid nullist

Näeme ka siit, et läbi aastate on ettevõte üha rohkem müünud ja äril läheb hästi.

# Klientide klasterdamine ehk segmenteerimine

Oletame, et me analüüsime antud andmestikku viimase tehingu kuupäeval, ehk siis täpselt 2018 aastavahetusel:

In [ ]:
raporteerimis_aeg = np.max(andmestik.tehingu_kuupäev)
raporteerimis_aeg

Järgmisena, arvutame sellised uued tunnused nagu HSV komponendid, ehk iga kliendi jaoks arvutame hiljutisuse, sageduse ning väärtuse. Me defineerime siin **hiljutisuse** kui kliendi kõigi tehingute seas minimaalse päevade arvu raporteerimisajast. **Sageduse** arvutame, lugedes mitu kirjet on andmestikus iga kliendi kohta, ehk kasutades `count` funktsiooni. **Väärtus** on iga kliendi ostusummma kokku.  

Teek `pandas` lubab andmeid aggregeerida mingi tunnuse järgi ja rakendada neile erinevaid funktsioone. Kasutasime seda funktsionaalsust varasemalt ka kuu ostusummade summeerimiseks.

In [ ]:
# defineerime funktsiooni, mis arvutab hiljutisuse
def tellimuse_hiljutisus(x):
    return min(raporteerimis_aeg - x)

In [ ]:
hsv = andmestik.groupby('kliendi_id').agg({'kliendi_id': 'count',
                                    'ostusumma':'sum',
                                   'tehingu_kuupäev': tellimuse_hiljutisus})
hsv.columns = ['sagedus', 'väärtus', "hiljutisus"]
hsv.hiljutisus = hsv.hiljutisus.dt.days
hsv.head()

In [ ]:
# uue andmestiku kirjeldav statistika
hsv.describe()

Õppematerjalis kirjeldasime väga lühidalt **k-keskmiste algoritmi**. Õnneks, ei pea me ise seda programmeerima vaid saame kasutada `sklearn` teeki, mis sisaldab K-keskmiste implementatsiooni. See tähendab, et hakkame otsima sarnaste klientide gruppe kasutades selleks eelpool loodud kolme tunnust.  

Enne seda peame me tunnuseid **skaleerima**. Mida see tähendab? Hetkel oleme me lugenud kliendi **sagedust** kordades (1 kuni 21), **väärtus** on pidev arv (0st kuni 190ni) ja **hiljutisus** on mõõdetud päevades (0st kuni 2144). Näeme, et erinevate tunnuste maksimaalsed (ja ka keskmised) väärtused erinevad lausa 100 korda. Seega, kui me arvutame ja võrdleme eukleidilist kaugust nende numbrite vahel, siis võrdleme nö õunu apelsinidega. Selleks, et meie tunnused viia sarnasele skaalale saame kasutada skaleerimist miinimumi ja maksimumiga:
$$x_{skaleeritud} = \frac{x_{originaal} - min(x)}{max(x) - min(x)} $$

Jällegi, `sklearn` teek saab meid ka siin aidata:

In [ ]:
skaleerimine = MinMaxScaler() # alguses tekitame skaleerimisobjekti
hsv_scaled = skaleerimine.fit_transform(hsv) # kasutame fit_transform et rakendada seda andmetel.
hsv_scaled

Nüüd on meie andmete kõik väärtused skaleeritud 0 ja 1 vahele. Saame nüüd kasutada k-keskmiste klasterdamisalgoritmi. Sarnaselt skaleerimise koodikastiga, alguses tekitame k-keskmiste objekti ja valime parameetrid ning siis rakendame seda defineeritud objekti meie andmete peale. Siin valime, et soovime leida 3 klastrit, sest antud algoritm nõuab sisendiks klastrite arvu, mida andmeteadlased määravad sisetunde ja katsetuse teel.

Tasub ka mainida, et parameetri `random_state` fikseerimine lubab samu tulemusi saada, kui seda analüüsi korrata. Kui see täpsustamata jätta, siis võivad igal optimeerimiskorral tulemused erineda, kuna algoritm, alustades erinevatest initisiaatoritest ja algsetest klastritest, võib lõpuks koonduda erinevatele tulemustele. Ülejaanud parameetrid kasutavad vaikimisi väärtusi.

Huvi korral vaata ka dokumentatsiooni k-keskmiste kohta siin: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html.



In [ ]:
kkeskmiste = KMeans(n_clusters=3, random_state=42)
ennustus_y = kkeskmiste.fit_predict(hsv_scaled)
ennustus_y[:10]

Väljundiks saime iga kliendi kohta klastri numbri kuhu see klient kuulub. Nüüd omistame selle meie andmetele:

In [ ]:
hsv['klaster'] = ennustus_y

Saame ka printida välja meie klastrite keskmed. Kuna andmed on skaleeritud, siis huvitav oleks vaadata seda ikkagi originaalskaalal. Seega kasutame vastupidist transformeerimist, kutsudes eelnevalt defineeritud objekti `skaleermine` ning funktsiooni `inverse_transform` mida saab tõlkida kui tagasi transformeerimist:

In [ ]:
# klastrite keskpunktid (kokku 3 punkti, kus väärtused on kujul [sagedus, väärtus, hiljutisus])
skaleerimine.inverse_transform(kkeskmiste.cluster_centers_)

Mida see siis näitab?

Esimese klastri puhul on sinna kuuluva kliendi keskmine **sagedus** (9.215) ja **väärtus** (78.198) kõrged, **hiljutisus** on madal (185.905). Need kliendid on meie "väärtuslikud kliendid".

Teine klaster on madala **sagedusega** (1.798), madala **väärtusega** (15.14) ja kõrge **hiljutisega** (1328.214). Need on ettevõtte "madala väärtusega" kliendid.

Ning viimane, kolmas klaster on vahepealne: seega "kõrge potentsiaaliga" kliendid.

Saame ka meie tulemusi visualiseerida. Näiteks hajuvusdiagrammil, kus saab valida kaks tunnust kolmest, nt. **väärtus** ja **hiljutisus**, ning värvida punkte vastavalt sellele, millisesse klastrisse nad kuuluvad:

In [ ]:
plt.figure(figsize=(8, 8))
sns.scatterplot(x="sagedus", y="hiljutisus", hue="klaster", data=hsv, palette='Set1')

Võid proovida ka ise visualiseerida hajuvusdiagramme **sageduse** ja **väärtuse**, ning **väärtuse** ja **hiljutisuse** vahel. Mida need joonised näitavad?

Tihti kasutame klasterdamist kui on olemas rohkem kui 3 tunnust. Kuid antud lihtsustatud näite puhul saame visualiseerida tulemust kasutades ka kõiki kolme tunnust korraga. Selleks kasutame 3-d hajuvusdiagrammi:

In [ ]:
# 3-d visualiseerimine (Valikuline)
plt.figure(figsize=(8, 8))
ax = plt.axes(projection='3d')
ax.scatter(hsv.hiljutisus, hsv.väärtus, hsv.sagedus, c=hsv.klaster)
ax.set_xlabel('hiljutisus')
ax.set_ylabel('väärtus')
ax.set_zlabel('sagedus')
#plt.savefig('3d_klastrid.png')

Vaatame, kui suured kliendigrupid me leidsime ja millised kliendid kuuluvad meie kõige väärtuslikemate klientide hulka.

In [ ]:
# kolme kliendigrupi suurused
hsv.klaster.value_counts()

In [ ]:
# millised kliendid kuuluvad esimesse gruppi (id-ga 0)
hsv[hsv.klaster == 0] # kliendid nähtaval kliendi_id veerus

Me saime teada, kes on meie kõige väärtuslikumad lemmikloomasõbrad, ning saame neile pakkuda soodustust.  

# Kokkuvõte
Antud praktikas me vaatlesime, kuidas andmestikus olevad tunnused välja näevad ja kuidas jagada olemasolevad kliendid segmentideks, et sihtida neid erinevalt.